In [23]:
import getpass
import os


if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")
else:
  print("OPENAI_API_KEY valid")

if not os.environ.get("LANGSMITH_API_KEY"):
  os.environ["LANGSMITH_API_KEY"] = getpass.getpass("Enter API key for LANGSMITH: ")
else:
  print("LANGSMITH_API_KEY valid")

OPENAI_API_KEY valid
LANGSMITH_API_KEY valid


In [24]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt
import requests
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display

In [25]:
import requests
from bs4 import BeautifulSoup

# Some websites need you to use proper headers when fetching them:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links and paragraph extraction
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
            # Extract all paragraph content
            self.paragraphs = [p.get_text(strip=True) for p in soup.find_all('p')]
            self.paragraph_text = "\n".join(self.paragraphs)
        else:
            self.text = ""
            self.paragraphs = []
            self.paragraph_text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"
    
    def get_paragraph_contents(self):
        """
        Returns all paragraph content from the webpage combined
        """
        return self.paragraph_text

In [26]:
huggingface = Website("https://huggingface.co/learn/llm-course/en/chapter1/4?fw=pt")
# Get all paragraph content
paragraph_content = huggingface.get_paragraph_contents()
# Or access the paragraphs list directly
paragraphs = huggingface.paragraphs

content = ""
for paragraph in paragraphs:
    content += paragraph
    content += '\n'

content = content.split('We will dive into these families in more depth later on.')[1]

In [ ]:
from transformers import pipeline

summarizer = pipeline("summarization")
summaried_content = summarizer(content)
summaried_content = summaried_content[0]['summary_text']

In [28]:
summaried_content

'this type of model develops a statistical understanding of the language it has been trained on . the general pretrained model then goes through a process calledtransfer learningorfine-tuning . this is because the output depends on the past and present inputs, but not the future ones .'

In [ ]:
help(pipeline("translation", model="VietAI/envit5-translation"))

In [44]:
from transformers import pipeline

translator = pipeline("translation", model="VietAI/envit5-translation")
prompt = f"translate English to Vietnamese: {summaried_content}"
translator(summaried_content, **{"max_length": 500})

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at VietAI/envit5-translation.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.
Device set to use 0


[{'translation_text': 'vi: Loại mô hình này phát triển một sự hiểu biết thống kê về ngôn ngữ mà nó đã được huấn luyện. mô hình được huấn luyện tổng quát sau đó trải qua một quá trình gọi là chuyển giao nhiễu học tập. điều này là do đầu ra phụ thuộc vào đầu vào quá khứ và hiện tại, nhưng không phụ thuộc vào tương lai.'}]